# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2018-2019 учебный год

Лектор: Илья Щуров

Семинаристы: Евгения Ческидова, Евгений Ковалев

Ассистенты: Константин Ваниев, Софья Дымченко

# Семинар 9

Сегодня мы:

- познакомимся с новым алгоритмом машинного обучения - решающим деревом

- напишем код для разбиения вершины

- <s>посадим дерево, вырастим сына, построим дом </s>

### Решающие деревья

<img src='tree_example.png'>

Сами по себе решающие деревья используются в машинном обучении относительно редко, однако очень распространены методы, основанные на их композиции - ансамблях (Random Forest, XGBoost, LightGBM).

##### Плюсы:

- интерпретируемость
- способность выделить самые важные признаки
- отсутствие потребности в серьезной предобработке данных

##### Минусы:

- склонность к переобучению
- неустойчивость - небольшие изменения в данных могут привести к сильному изменению в структуре дерева
- эвристичность обучения - как оптимизировать?

#####  Линейная модель vs "деревянная" модель (основанная на решающих деревьях):

- когда данные хорошо линейно разделимы, линейная модель лучше

- когда данные плохо линейно разделимы (много сложных нелинейных зависимостей в данных), "деревянная" модель лучше

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)

### Переобучение

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

In [ ]:
np.random.seed(13)
data_x = np.random.normal(size=(100, 2))
data_y = (data_x[:, 0] ** 2 + data_x[:, 1] ** 2) ** 0.5 # 
plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor(random_state=13)
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='spring')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring', edgecolor='k')
plt.show()

Посмотрим, как будет выглядеть разделение плоскости в зависимости от максимальной глубины дерева и минимального числа объектов в листе.

In [ ]:
plt.figure(figsize=(14, 14))
for i, max_depth in enumerate([2, 4, None]):
    for j, min_samples_leaf in enumerate([15, 5, 1]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=13)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        plt.subplot2grid((3, 3), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='spring')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='spring', edgecolor='k')
        plt.title('max_depth=' + str(max_depth) + ' | min_samples_leaf=' + str(min_samples_leaf))

### Неустойчивость

Посмотрим, как будет меняться структура дерева, если брать для обучения разные 90%-ые подвыборки исходной выборки.

In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=13)

    indecies = np.random.randint(data_x.shape[0], size=int(data_x.shape[0] * 0.9))
    clf.fit(data_x[indecies], data_y[indecies])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='winter')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter', edgecolor='k')

### Практика

In [ ]:
boston = load_boston()

In [ ]:
print(boston['DESCR'])

In [ ]:
boston.keys()

In [ ]:
boston['feature_names']

In [ ]:
boston['data'].shape

In [ ]:
plt.hist(boston['target'])
plt.show()

In [ ]:
data_train, data_test, y_train, y_test = train_test_split(boston['data'], boston['target'], random_state=13)

In [ ]:
X_train = pd.DataFrame(data_train, columns=boston['feature_names'])
X_test = pd.DataFrame(data_test, columns=boston['feature_names'])
X_train['target'] = y_train

In [ ]:
X_train.head()

In [ ]:
X_train.shape, X_test.shape

$R_m$ - множество объектов в разбиваемой вершине, $i$ - номер признака, по которому происходит разбиение, $t$ - порог разбиения.

Качество:

$$
Q(R_m, i, t) = H(R_m) - \frac{|R_\ell|}{|R_m|}H(R_\ell) - \frac{|R_r|}{|R_m|}H(R_r)
$$

$R_\ell$ - множество объектов в левом поддереве, $R_r$ - множество объектов в правом поддереве.

$H(R)$ - критерий информативности, с помощью которого можно оценить качество распределения целевой переменной среди объектов множества $R$.

_Вопрос. Что мы хотим сделать с $H(R)$ - минимизировать или максимизировать? А $Q(R_m, i, t)$?_

_Вопрос. Что можно использовать в качестве критерия информативности для регрессии? *А для классификации?_

_Реализуйте подсчет качества разбиения._

In [ ]:
def H(R):
    # your code here
    return


def split_node(R_m, feature, t):
    # your code here
    return R_l, R_r


def quality(R_m, feature, t):
    # your code here
    return Q

_Переберите все возможные разбиения выборки по одному из признаков и постройте график качества разбиения в зависимости от значения порога._

In [ ]:
# your code here

_Напишите функцию, находящую оптимальное разбиение данной вершины по данному признаку._

In [ ]:
def get_optimal_split(R_m, feature):
    # your code here
    return t, Q_arr

_Постройте график качества разбиения (в зависимости от количества объектов в левом поддереве) для каждого из признаков. Найдите признак, показывающий наилучшее качество. Какой это признак? Каков порог разбиения и значение качества? Постройте график качества разбиения для данного признака в зависимости от значения порога._

In [ ]:
# your code here

_Изобразите разбиение визуально. Для этого постройте диаграмму рассеяния целевой переменной в зависимости от значения найденного признака. Далее изобразите вертикальную линию, соответствующую разбиению. Почему это разбиение лучшее? Как вы можете интерпретировать результат?_

In [ ]:
# your code here